In [1]:
import geopandas as gpd
import pandas as pd
%matplotlib inline

In [2]:
TILES_SHP = '/storage/lidar/clackamol_2013/processed/vectors/tiles.shp'
FOOTPRINT_SHP = '/storage/lidar/clackamol_2013/processed/vectors/footprint.shp'

In [3]:
gdf = gpd.read_file(TILES_SHP)

In [4]:
gdf['mil_points'] = gdf['num_points'] / 1000000.

In [5]:
gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2057 entries, 0 to 2056
Data columns (total 13 columns):
file_name     2057 non-null object
version       2057 non-null object
num_points    2057 non-null int64
point_type    2057 non-null int64
point_size    2057 non-null int64
min_x         2057 non-null float64
max_x         2057 non-null float64
min_y         2057 non-null float64
max_y         2057 non-null float64
min_z         2057 non-null float64
max_z         2057 non-null float64
geometry      2057 non-null object
mil_points    2057 non-null float64
dtypes: float64(7), int64(3), object(3)
memory usage: 209.0+ KB


In [6]:
buffered = gdf.drop(['file_name', 'point_size', 'point_type', 'num_points'], axis=1)
buffered.geometry = gdf.buffer(0.01) # buffer by 1cm

In [7]:
try:
    union = gpd.GeoDataFrame(geometry=list(buffered.unary_union), crs=buffered.crs)
except TypeError: # line above will fail if there is only one polygon for the footprint
    union = gpd.GeoDataFrame(geometry=[buffered.unary_union], crs=buffered.crs)
    
union['footprint_id'] = union.index + 1

In [8]:
buffered = gpd.tools.sjoin(buffered, union, how='left').drop('index_right', axis=1)

In [9]:
aggfuncs={'mil_points':'sum', 
          'version':'first', 
          'min_x':'min',
          'min_y':'min',
          'min_z':'min',
          'max_x':'max',
          'max_y':'max',
          'max_z':'max'
         }

In [10]:
dissolved = buffered.dissolve(by='footprint_id', aggfunc=aggfuncs)

In [11]:
dissolved.head()

,geometry,mil_points,version,min_x,min_y,min_z,max_x,max_y,max_z
footprint_id,,,,,,,,,
1,"POLYGON ((552999.99 4968086.9, 552999.99 49684...",19603.208297,LAS 1.2,507899.54,4960911.15,45.11,600335.49,5024805.63,1551.06


In [12]:
dissolved.to_file(FOOTPRINT_SHP)